In [1]:
import numpy as np
import pandas as pd
import typing

from rectools import Columns
from rectools.metrics import Precision, Recall, MeanInvUserFreq, Serendipity, Accuracy, F1Beta
from rectools.metrics.ranking import MAP, MRR, NDCG
from rectools.models import RandomModel, PopularModel
from rectools.model_selection import TimeRangeSplitter

import sys  
sys.path.insert(1, '../')  # to make rec_sys visible

from rec_sys.metrics import CVMetrics
from rec_sys.visual_analysis import VisualAnalysis

d:\Univer\ITMO\1 term\recsys\recsys-itmo\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Init metrics

In [2]:
n_splits = 3

models = {
    "random": RandomModel(random_state=47),
    "popular_int": PopularModel(popularity='n_interactions'),
}

def add_metrics(metrics: typing.Dict, name: str, metric: typing.Callable):
    metrics[f"{name}@1"] = metric(k=1)
    metrics[f"{name}@5"] = metric(k=5)
    metrics[f"{name}@10"] = metric(k=10)


metrics = {}
add_metrics(metrics, "Precision", Precision)
add_metrics(metrics, "Recall", Recall)
add_metrics(metrics, "Accuracy", Accuracy)
add_metrics(metrics, "F1", F1Beta)
add_metrics(metrics, "MAP", MAP)
add_metrics(metrics, "MRR", MRR)
add_metrics(metrics, "NDCG", NDCG)
add_metrics(metrics, "MeanInvUserFreq", MeanInvUserFreq)
add_metrics(metrics, "Serendipity", Serendipity)

splitter = TimeRangeSplitter(
    test_size="7D",
    n_splits=n_splits,
    filter_already_seen=True,
    filter_cold_items=True,
    filter_cold_users=True,
)

In [3]:
metrics = CVMetrics(
    models=models,
    metrics=metrics,
    splitter=splitter,
    k=10
)

# Read the data

In [4]:
interactions_df = pd.read_csv(
    "../data/interactions.csv", 
    sep=",",
    names=[Columns.User, Columns.Item, Columns.Datetime, 'total_dur', 'watched_pct'],
)

interactions_df = interactions_df.iloc[1:]
interactions_df = interactions_df[[Columns.User, Columns.Item, Columns.Datetime]]
interactions_df[Columns.Weight] = 1
interactions_df[Columns.Score] = 1
interactions_df[Columns.Item] = interactions_df[Columns.Item].astype("int")
interactions_df[Columns.User] = interactions_df[Columns.User].astype("int")

print(interactions_df.shape)
interactions_df.head()

C:\Users\-\AppData\Local\Temp\ipykernel_15856\3773194343.py:1: DtypeWarning: Columns (0,1,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactions_df = pd.read_csv(


(5476251, 5)


,user_id,item_id,datetime,weight,score
1,176549,9506,2021-05-11,1,1
2,699317,1659,2021-05-29,1,1
3,656683,7107,2021-05-09,1,1
4,864613,7638,2021-07-05,1,1
5,964868,9506,2021-04-30,1,1


In [5]:
items_df = pd.read_csv(
    "../data/items.csv", 
    sep=",",
)
items_df.head()

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."
3,7868,film,45 лет,45 Years,2015.0,"драмы, зарубежные, мелодрамы",Великобритания,NaN,16.0,NaN,Эндрю Хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю..."
4,16268,film,Все решает мгновение,NaN,1978.0,"драмы, спорт, советские, мелодрамы",СССР,NaN,12.0,Ленфильм,Виктор Садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж..."


# Calculate metrics

In [6]:
metrics.calculate_metrics(interactions_df)

100%|██████████| 3/3 [03:17<00:00, 65.97s/it]


In [7]:
metrics.print_metrics()

In [8]:
metrics.models

{'random': <rectools.models.random.RandomModel at 0x1a6a07c9540>,
 'popular_int': <rectools.models.popular.PopularModel at 0x1a6a07c94b0>}

# Visual Analysis

In [9]:
visual_analysis = VisualAnalysis(
    trained_model=metrics.models["random"],
    interactions_df=interactions_df,
    user_ids=np.array([666262, 672861, 955527]),
    item_data=items_df,
).visualize()

In [10]:
for user in visual_analysis:
    print(f'Пользователь: {user}')
    print('Рекомендации:')
    display(visual_analysis[user]["recommendations"])
    print('История просмотров:')
    display(visual_analysis[user]["user_history"])
    print('\n')

Пользователь: 666262
Рекомендации:


,title,genres,user_views,total_views,score,rank
0,И шарик вернется,"русские, мелодрамы",0,26,10,1
1,Малефисента,"семейное, фэнтези, приключения, мелодрамы",0,1956,9,2
2,Баксы,"драмы, фэнтези",0,9,8,3
3,Маруся 2. Трудные взрослые,"русские, мелодрамы",0,5,7,4
4,Спайды,"фантастика, зарубежные, триллеры, детективы",0,22,6,5
5,Республика Сары,драмы,0,46,5,6
6,Стычка в ночи,"драмы, зарубежные, фильм-нуар, мелодрамы",0,3,4,7
7,Невероятная история о гигантской груше,"мультфильм, приключения",0,150,3,8
8,Таймлесс 2: Сапфировая книга,приключения,0,7,2,9
9,Беги,"триллеры, криминал, комедии",0,40,1,10


История просмотров:


,title,genres,datetime,user_views,total_views
0,Последний викинг,"боевики, историческое, приключения",2021-05-12,1,746
1,Робин Гуд: Начало,"боевики, триллеры, приключения",2021-05-12,1,485
2,Томирис,"боевики, драмы, историческое, военные",2021-05-14,1,10370




Пользователь: 672861
Рекомендации:


...

,title,genres,datetime,user_views,total_views
0,Красавица и чудовище,"драмы, фэнтези, музыкальные",2021-04-27,1,1083
1,Он – дракон,фэнтези,2021-05-04,1,643




Пользователь: 955527
Рекомендации:


,title,genres,user_views,total_views,score,rank
0,Легенда о Нараяме,драмы,0,186,10,1
1,Костер в белой ночи,"катастрофы, семейное, советские, боевики",0,1,9,2
2,Покемон Фильм: Курем против Меча справедливости,"аниме, приключения, зарубежные, семейное, фэнтези",0,2,8,3
3,Без особых примет,"зарубежные, триллеры",0,1,7,4
4,Я больше не верю в любовь,"драмы, зарубежные",0,2,6,5
5,Семеро солдатиков,семейное,0,13,5,6
6,Просто о важном. Про Миру и Гошу,"развлекательные, развитие, мультфильмы",0,48,4,7
7,Влюблённые женщины,драмы,0,167,3,8
8,"Карты, деньги, два ствола",комедии,0,61,2,9
9,Максим Перепелица,"советские, комедии",0,14,1,10


История просмотров:


,title,genres,datetime,user_views,total_views
0,Стань легендой! Бигфут Младший,"мультфильм, фэнтези, приключения, комедии",2021-06-02,1,1587
1,Пеле: Рождение легенды,"драмы, спорт, биография",2021-05-04,1,945
2,Лобановский навсегда,"спорт, биография, документальное",2021-06-02,1,683
3,Диего Марадона,"спорт, биография, документальное",2021-06-02,1,691
